# Naive Approach to ARIMA modelling using MSE values as loss function and Fourier time series approximation to handle seasonality

In [1]:
!pip install pmdarima

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import datetime, timedelta
from itertools import product
from pandas.plotting import register_matplotlib_converters
from pmdarima import auto_arima, ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, plot_predict
from statsmodels.tsa.stattools import acf, pacf
# from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

register_matplotlib_converters()

In [3]:
# TESTING = False
# item_count, store_count = 50, 10
# forecast_range = 90

# df = pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/train.csv')
# df_eval =pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/test.csv')

# df["date"] = pd.to_datetime(df["date"])
# df_eval["date"] = pd.to_datetime(df["date"])
# df

In [4]:
# Test best parameters before submission
TESTING = True
item_count, store_count = 5, 5
forecast_range = 90
n_harm = 100

df = pd.read_csv('../data/train.csv')
df_eval =pd.read_csv('../data/test.csv')

df["date"] = pd.to_datetime(df["date"])
df_eval["date"] = pd.to_datetime(df_eval["date"])
df, df_eval

(             date  store  item  sales
 0      2013-01-01      1     1     13
 1      2013-01-02      1     1     11
 2      2013-01-03      1     1     14
 3      2013-01-04      1     1     13
 4      2013-01-05      1     1     10
 ...           ...    ...   ...    ...
 912995 2017-12-27     10    50     63
 912996 2017-12-28     10    50     59
 912997 2017-12-29     10    50     74
 912998 2017-12-30     10    50     62
 912999 2017-12-31     10    50     82
 
 [913000 rows x 4 columns],
           id       date  store  item
 0          0 2018-01-01      1     1
 1          1 2018-01-02      1     1
 2          2 2018-01-03      1     1
 3          3 2018-01-04      1     1
 4          4 2018-01-05      1     1
 ...      ...        ...    ...   ...
 44995  44995 2018-03-27     10    50
 44996  44996 2018-03-28     10    50
 44997  44997 2018-03-29     10    50
 44998  44998 2018-03-30     10    50
 44999  44999 2018-03-31     10    50
 
 [45000 rows x 4 columns])

In [5]:
def fourier_extrapolation(x, n_predict, n_harm=50):
    n = x.size
    t = np.arange(0, n)
    p = np.polyfit(t, x, 1)         # Find linear trend in x
    x_detrended = x - p[0] * t      # Detrended x
    x_freq_domain = np.fft.fft(x_detrended)  # Detrended x in the frequency domain
    f = np.fft.fftfreq(n)              # Frequencies
    indexes = list(range(n))
    # Sort indexes by frequency, lower -> higher
    indexes.sort(key=lambda i: np.absolute(f[i]))

    t = np.arange(0, n + n_predict)
    restored_signal = np.zeros(t.size)
    for i in indexes[:1 + n_harm * 2]:
        amplitude = np.absolute(x_freq_domain[i]) / n   # Amplitude
        phase = np.angle(x_freq_domain[i])             # Phase
        restored_signal += amplitude * np.cos(2 * np.pi * f[i] * t + phase)
        
    restored_signal = restored_signal + p[0] * t
    restored_signal = pd.Series(restored_signal, 
                               index=pd.date_range(
                                   start=x.index[0],
                                   periods=len(x) + n_predict,
                                   freq='D'
                               )
                              )
    
    in_sample_signal = restored_signal[:n]
    future_signal = restored_signal[n:]

    return in_sample_signal, future_signal

In [6]:
series_dict = dict()
actual_series_dict = dict()   # For testing
in_sample_fourier_dict = dict()
future_fourier_dict = dict()

for (iid, sid) in product(range(1, item_count+1), range(1, store_count+1)):
    df_t = df[(df["item"] == iid) & (df["store"] == sid)]
    
    if not TESTING:
        series_t = pd.Series(df_t["sales"].values, df_t["date"])
    else:
        series_t = pd.Series(df_t["sales"][:-forecast_range].values,
                             df_t["date"][:-forecast_range])
        actual_series_dict[(iid, sid)] = pd.Series(df_t["sales"][-forecast_range:].values,
                                                   df_t["date"][-forecast_range:])
        
    iss, fs = fourier_extrapolation(series_t, forecast_range, n_harm)
    
    series_dict[(iid, sid)] = series_t
    in_sample_fourier_dict[(iid, sid)] = iss
    future_fourier_dict[(iid, sid)] = fs

In [7]:
fitted_model_dict = dict()
pred_df = pd.DataFrame(columns=["date", "store", "item", "sales"])
pred_series_dict = dict()

# Requires sid to iterate first then iid to iterate
for (sid, iid) in product(range(1, store_count+1), range(1, item_count+1)):
    print(sid, iid)
    model = ARIMA((5, 0, 5), suppress_warnings=True, enforce_stationarity=False)
    model = model.fit(series_dict[iid, sid],
                      np.array(in_sample_fourier_dict[iid, sid]).reshape(-1, 1)
                     )
#     model = auto_arima(series_dict[iid, sid],
#                        np.array(in_sample_fourier_dict[iid, sid]).reshape(-1,1)
#                       )
    fitted_model_dict[(iid, sid)] = model
    
    pred = model.predict(forecast_range, np.array(future_fourier_dict[(iid, sid)]).reshape(-1, 1))
    pred = np.round(pred).astype(int)
    
    pred_series_dict[(iid, sid)] = pred
    pred = pd.DataFrame({
        "date": pred.index,
        "store": sid,
        "item": iid,
        "sales": pred.values
    })
    pred_df = pd.concat([pred_df, pred], axis=0)


1 1


/tmp/ipykernel_206800/2590526100.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pred_df = pd.concat([pred_df, pred], axis=0)


1 2
1 3
1 4
1 5
2 1
2 2
2 3
2 4
2 5
3 1
3 2
3 3
3 4
3 5
4 1
4 2
4 3
4 4
4 5
5 1
5 2
5 3
5 4
5 5


In [8]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

if TESTING:
    score_pred = np.array([])
    score_actual = np.array([])
    for (sid, iid) in product(range(1, store_count+1), range(1, item_count+1)):
        score_pred = np.concatenate([score_pred, pred_series_dict[(iid, sid)]])
        score_actual = np.concatenate([score_actual, actual_series_dict[(iid, sid)]])
    print(f"Score is {smape(score_actual, score_pred)}")
else:
    # Write submission csv
    df_submit = df_eval.merge(pred_df, on = ["date", "store", "item"]).sort_values(by="id")[["id", "sales"]]
    df_submit.to_csv("submission.csv", index=False)

Score is 24.126505270332867
